# Rebuild All Events
    This script builds/rebuilds all the events of Live Mouse Tracker
    You can also create your own events and add them to this process
    
    Approximate time is 1h of computation for a 24h database of 4 animals


# Parameters:
    This will compute automatically events for your database, and adjust settings for you.
    
    If you remove the automatic settings, you can force parameters:
        Set minT and maxT to process the database.
        Set windowT to divide the computation load in segment of that duration. Default value is 1 day.

In [1]:
import sys
sys.path.insert(1, "../")
from lmtanalysis.Measure import *

# set this to false if you want to set manual parameters.
automaticSettings = True

# Manual parameters:

''' minT and maxT to process the analysis (expressed in frame) '''
minT = 0
maxT = 3*oneDay
''' time window to compute the events (in frame). '''
windowT = 1*oneDay
''' speed up process '''
USE_CACHE_LOAD_DETECTION_CACHE = True

# Run this section to compute your database
    You will be prompt to provide a database

In [2]:
''' Created on 26 march 2019 @author: Fab '''

import sys
sys.path.insert(1, "../")
import sqlite3
from lmtanalysis.Animal import *
import matplotlib.pyplot as plt
from lmtanalysis.Event import *
from lmtanalysis.Measure import *
from lmtanalysis import BuildEventTrain3, BuildEventTrain4, BuildEventTrain2, BuildEventFollowZone, BuildEventRear5, BuildEventFloorSniffing,\
    BuildEventSocialApproach, BuildEventSocialEscape, BuildEventApproachContact,BuildEventOralOralContact,\
    BuildEventApproachRear, BuildEventGroup2, BuildEventGroup3, BuildEventGroup4, BuildEventOralGenitalContact, \
    BuildEventStop, BuildEventWaterPoint, \
    BuildEventMove, BuildEventGroup3MakeBreak, BuildEventGroup4MakeBreak,\
    BuildEventSideBySide, BuildEventSideBySideOpposite, BuildEventDetection,\
    BuildDataBaseIndex, BuildEventWallJump, BuildEventSAP,\
    BuildEventOralSideSequence, CheckWrongAnimal,\
    CorrectDetectionIntegrity, BuildEventNest4, BuildEventNest3, BuildEventFight, BuildEventGetAway
    
from psutil import virtual_memory

from tkinter.filedialog import askopenfilename
from lmtanalysis.TaskLogger import TaskLogger
import sys
import traceback
from lmtanalysis.FileUtil import getFilesToProcess
from lmtanalysis.EventTimeLineCache import flushEventTimeLineCache,\
    disableEventTimeLineCache


from lmtanalysis.EventTimeLineCache import EventTimeLineCached

def getNumberOfFrames(file):
    
    connection = sqlite3.connect( file )
    c = connection.cursor() 
    query = "SELECT MAX(FRAMENUMBER) FROM FRAME";
    c.execute( query )
    result = c.fetchall()
    numberOfFrames = int( result[0][0] )
    connection.close()
    
    return numberOfFrames

class FileProcessException(Exception):
    pass

eventClassList = [                  
                  BuildEventOralOralContact,
                  BuildEventOralGenitalContact,
                  BuildEventSideBySide,
                  BuildEventSideBySideOpposite,
                  BuildEventTrain2,                  
                  BuildEventTrain3,
                  BuildEventTrain4,
                  BuildEventMove,
                  BuildEventFollowZone,
                  BuildEventRear5,
                  BuildEventSocialApproach,
                  BuildEventGetAway,
                  BuildEventSocialEscape,
                  BuildEventApproachRear,
                  BuildEventGroup2,
                  BuildEventGroup3,
                  BuildEventGroup4,
                  BuildEventGroup3MakeBreak,
                  BuildEventGroup4MakeBreak,
                  BuildEventStop,
                  BuildEventWaterPoint,
                  BuildEventApproachContact,
                  BuildEventWallJump,
                  BuildEventSAP,
                  BuildEventOralSideSequence,
                  BuildEventNest3,
                  BuildEventNest4
                   ]

def flushEvents( connection ):
    
    print("Flushing events...")

    for ev in eventClassList:
        
        chrono = Chronometer( "Flushing event " + str(ev) )
        ev.flush( connection );      
        chrono.printTimeInS()
    

def processTimeWindow( connection, currentMinT , currentMaxT ):
    
    CheckWrongAnimal.check( connection, tmin=currentMinT, tmax=currentMaxT )
    
    # Warning: enabling this process (CorrectDetectionIntegrity) will alter the database permanently
    # CorrectDetectionIntegrity.correct( connection, tmin=0, tmax=maxT )
                            
    BuildEventDetection.reBuildEvent( connection, file, tmin=currentMinT, tmax=currentMaxT )

    animalPool = None
    
    flushEventTimeLineCache()
    
    if ( USE_CACHE_LOAD_DETECTION_CACHE ):
        print("Caching load of animal detection...")
        animalPool = AnimalPool( )
        animalPool.loadAnimals( connection )
        animalPool.loadDetection( start = currentMinT, end = currentMaxT )
        print("Caching load of animal detection done.")

    for ev in eventClassList:
        
        chrono = Chronometer( str( ev ) )
        ev.reBuildEvent( connection, file, tmin=currentMinT, tmax=currentMaxT, pool = animalPool )        
        chrono.printTimeInS()

def process( file ):

    print(file)
    
    if automaticSettings:
        print("Automatic settings.")
        windowT = 1*oneDay
        minT = 0
        maxT = getNumberOfFrames( file )
        print ( "Auto max set to" , maxT , "frames")
        
    
    chronoFullFile = Chronometer("File " + file )
    
    connection = sqlite3.connect( file )
    
    BuildDataBaseIndex.buildDataBaseIndex( connection, force=False )
        
    # build sensor data
    animalPool = AnimalPool( )
    animalPool.loadAnimals( connection )
    animalPool.buildSensorData(file)
    
    currentT = minT

    try:

        flushEvents( connection )
        
        while currentT < maxT:
                        
            currentMinT = currentT
            currentMaxT = currentT+ windowT
            if ( currentMaxT > maxT ):
                currentMaxT = maxT
                
            chronoTimeWindowFile = Chronometer("File "+ file+ " currentMinT: "+ str(currentMinT)+ " currentMaxT: " + str(currentMaxT) );
            processTimeWindow( connection, currentMinT, currentMaxT )    
            chronoTimeWindowFile.printTimeInS()
            
            currentT += windowT

                        

        print("Full file process time: ")
        chronoFullFile.printTimeInS()
        
        TEST_WINDOWING_COMPUTATION = False
        
        if ( TEST_WINDOWING_COMPUTATION ):
                
            print("*************")
            print("************* TEST START SECTION")
            print("************* Test if results are the same with or without the windowing.")
            
            # display and record to a file all events found, checking with rolling idA from None to 4. Save nbEvent and total len
            
            eventTimeLineList = []
            
            eventList = getAllEvents( connection )
            file = open("outEvent"+str(windowT)+".txt","w")  
            file.write( "Event name\nnb event\ntotal duration" )
            
            for eventName in eventList:
                for idAnimalA in range( 0,5 ):                
                        idA = idAnimalA 
                        if idA == 0:
                            idA = None
                        timeLine = EventTimeLineCached( connection, file, eventName, idA,  minFrame=minT, maxFrame=maxT )
                        eventTimeLineList.append( timeLine )
                        file.write( timeLine.eventNameWithId+"\t"+str(len(timeLine.eventList))+"\t"+str(timeLine.getTotalLength())+"\n" )            
            
            file.close() 
    
            #plotMultipleTimeLine( eventTimeLineList )
            
            print("************* END TEST")
        
        
    except:
        
        exc_type, exc_value, exc_traceback = sys.exc_info()
        lines = traceback.format_exception(exc_type, exc_value, exc_traceback)
        error = ''.join('!! ' + line for line in lines)
        
        t = TaskLogger( connection )
        t.addLog( error )
        
        print( error, file=sys.stderr ) 
        
        raise FileProcessException()
            
def getAllEvents( connection ):
    
    query = "select name from event group by name order by name"
    c = connection.cursor()     
    c.execute( query )    
    all_rows = c.fetchall()        
    header = [ "Name" ]
    data =[]    
    for row in all_rows:
        data.append( row[0] )
    return data        

print("Code launched.")

mem = virtual_memory()
availableMemoryGB = mem.total / 1000000000
print( "Total memory on computer: (GB)", availableMemoryGB ) 

if availableMemoryGB < 10:
    print( "Not enough memory to use cache load of events.")
    disableEventTimeLineCache()

print("A window is popping (maybe hidden) asking for files to process...")
files = getFilesToProcess()

chronoFullBatch = Chronometer("Full batch" )    

if ( files != None ):

    for file in files:
        try:
            print ( "Processing file" , file )
            process( file )
        except FileProcessException:
            print ( "STOP PROCESSING FILE " + file , file=sys.stderr  )

        flushEventTimeLineCache()

chronoFullBatch.printTimeInS()
print( "*** ALL JOBS DONE ***")

Code launched.
Total memory on computer: (GB) 17.078222848
A window is popping (maybe hidden) asking for files to process...
Processing file E:/lmt/base validation/20180110_validation_4_ind_Experiment_6644/20180110_validation_4_ind_Experiment_6644_NEW.sqlite
E:/lmt/base validation/20180110_validation_4_ind_Experiment_6644/20180110_validation_4_ind_Experiment_6644_NEW.sqlite
Automatic settings.
Auto max set to 23890 frames
Creating lmtanalysis indexes...
CREATE INDEX `animalIndex` ON `ANIMAL` (`ID` );
index animalIndex already exists
CREATE INDEX `detectionIndex` ON `DETECTION` (`ID` ASC,`FRAMENUMBER` ASC);
index detectionIndex already exists
CREATE INDEX `detetIdIndex` ON `DETECTION` (`ID` ASC);
index detetIdIndex already exists
CREATE INDEX `detframenumberIndex` ON `DETECTION` (`FRAMENUMBER` ASC);
index detframenumberIndex already exists
CREATE INDEX `eventEndFrameIndex` ON `EVENT` (`ENDFRAME` ASC);
index eventEndFrameIndex already exists
CREATE INDEX `eventIndex` ON `EVENT` (`ID` ASC

Number of event deleted: 994 
Deleting event Stop isolated (idA:None,idB:None,idC:None,idD:None) from base...
DELETE FROM EVENT WHERE NAME="Stop isolated"
Number of event deleted: 2547 
[Chrono  Flushing event <module 'lmtanalysis.BuildEventStop' from '..\\lmtanalysis\\BuildEventStop.py'>  ]  0.6562397480010986  seconds
Deleting event Water Zone (idA:None,idB:None,idC:None,idD:None) from base...
DELETE FROM EVENT WHERE NAME="Water Zone"
Number of event deleted: 82 
[Chrono  Flushing event <module 'lmtanalysis.BuildEventWaterPoint' from '..\\lmtanalysis\\BuildEventWaterPoint.py'>  ]  0.09373664855957031  seconds
Deleting event Approach contact (idA:None,idB:None,idC:None,idD:None) from base...
DELETE FROM EVENT WHERE NAME="Approach contact"
Number of event deleted: 1396 
[Chrono  Flushing event <module 'lmtanalysis.BuildEventApproachContact' from '..\\lmtanalysis\\BuildEventApproachContact.py'>  ]  0.24999690055847168  seconds
Deleting event WallJump (idA:None,idB:None,idC:None,idD:None

Oral-oral Contact
Event Oral-oral Contact created. eventNameWithId = Oral-oral Contact idA:2 idB:1 idC:None idD:None loadEvent: False
Number of event:  64
Mean length of event:  5.0
first event frame:  1065
Keep previous entry.
Saving timeLine: Oral-oral Contact ( 64 events )
Oral-oral Contact
Event Oral-oral Contact created. eventNameWithId = Oral-oral Contact idA:2 idB:3 idC:None idD:None loadEvent: False
Number of event:  40
Mean length of event:  5.55
first event frame:  1248
Keep previous entry.
Saving timeLine: Oral-oral Contact ( 40 events )
Oral-oral Contact
Event Oral-oral Contact created. eventNameWithId = Oral-oral Contact idA:2 idB:4 idC:None idD:None loadEvent: False
Number of event:  31
Mean length of event:  5.354838709677419
first event frame:  945
Keep previous entry.
Saving timeLine: Oral-oral Contact ( 31 events )
Oral-oral Contact
Event Oral-oral Contact created. eventNameWithId = Oral-oral Contact idA:3 idB:1 idC:None idD:None loadEvent: False
Number of event:  54


Number of event:  12
Mean length of event:  4.916666666666667
first event frame:  4041
Keep previous entry.
Saving timeLine: Side by side Contact ( 12 events )
Side by side Contact
Event Side by side Contact created. eventNameWithId = Side by side Contact idA:3 idB:1 idC:None idD:None loadEvent: False
Number of event:  27
Mean length of event:  6.518518518518518
first event frame:  2741
Keep previous entry.
Saving timeLine: Side by side Contact ( 27 events )
Side by side Contact
Event Side by side Contact created. eventNameWithId = Side by side Contact idA:3 idB:2 idC:None idD:None loadEvent: False
Number of event:  17
Mean length of event:  5.235294117647059
first event frame:  6131
Keep previous entry.
Saving timeLine: Side by side Contact ( 17 events )
Side by side Contact
Event Side by side Contact created. eventNameWithId = Side by side Contact idA:3 idB:4 idC:None idD:None loadEvent: False
Number of event:  21
Mean length of event:  4.238095238095238
first event frame:  4044
Keep

Oral-genital Contact  Id( 4 , 3 , None , None ) Loaded ( 31  records loaded in  0.04687786102294922 S )
Caching eventTimeLine
Train2
Event Train2 created. eventNameWithId = Train2 idA:1 idB:2 idC:None idD:None loadEvent: False
no event
Keep previous entry.
Saving timeLine: Train2 ( 0 events )
Train2
Event Train2 created. eventNameWithId = Train2 idA:1 idB:3 idC:None idD:None loadEvent: False
Number of event:  1
Mean length of event:  13.0
first event frame:  2765
Keep previous entry.
Saving timeLine: Train2 ( 1 events )
Train2
Event Train2 created. eventNameWithId = Train2 idA:1 idB:4 idC:None idD:None loadEvent: False
no event
Keep previous entry.
Saving timeLine: Train2 ( 0 events )
Train2
Event Train2 created. eventNameWithId = Train2 idA:2 idB:1 idC:None idD:None loadEvent: False
no event
Keep previous entry.
Saving timeLine: Train2 ( 0 events )
Train2
Event Train2 created. eventNameWithId = Train2 idA:2 idB:3 idC:None idD:None loadEvent: False
no event
Keep previous entry.
Saving 

...

Group3
Event Group3 created. eventNameWithId = Group3 idA:4 idB:2 idC:3 idD:None loadEvent: False
Number of event:  17
Mean length of event:  9.941176470588236
first event frame:  4074
Keep previous entry.
Saving timeLine: Group3 ( 17 events )
Group3
Event Group3 created. eventNameWithId = Group3 idA:4 idB:3 idC:1 idD:None loadEvent: False
Number of event:  20
Mean length of event:  3.7
first event frame:  2689
Keep previous entry.
Saving timeLine: Group3 ( 20 events )
Group3
Event Group3 created. eventNameWithId = Group3 idA:4 idB:3 idC:2 idD:None loadEvent: False
Number of event:  17
Mean length of event:  9.941176470588236
first event frame:  4074
Keep previous entry.
Saving timeLine: Group3 ( 17 events )
INSERT INTO LOG( process,version,date,tmin,tmax) VALUES ( 'Build Event Group 3','0','2019-04-01 15:18:15','0','23890' );
Rebuild event finished.
[Chrono  <module 'lmtanalysis.BuildEventGroup3' from '..\\lmtanalysis\\BuildEventGroup3.py'>  ]  0.9062433242797852  seconds
Loading anim

Group3  Id( 4 , 1 , 2 , None ) Loaded ( 21  records loaded in  0.07813310623168945 S )
Caching eventTimeLine
Group3  Id( 4 , 1 , 3 , None ) Loaded ( 20  records loaded in  0.07813310623168945 S )
Caching eventTimeLine
Group3  Id( 4 , 2 , 1 , None ) Loaded ( 21  records loaded in  0.07812190055847168 S )
Caching eventTimeLine
Group3  Id( 4 , 2 , 3 , None ) Loaded ( 17  records loaded in  0.09374618530273438 S )
Caching eventTimeLine
Group3  Id( 4 , 3 , 1 , None ) Loaded ( 20  records loaded in  0.07812952995300293 S )
Caching eventTimeLine
Group3  Id( 4 , 3 , 2 , None ) Loaded ( 17  records loaded in  0.09376859664916992 S )
Caching eventTimeLine
Number of event:  19
Mean length of event:  1.0
first event frame:  9549
Keep previous entry.
Saving timeLine: Group 3 make ( 19 events )
Number of event:  19
Mean length of event:  1.0
first event frame:  4500
Keep previous entry.
Saving timeLine: Group 3 break ( 19 events )
Number of event:  31
Mean length of event:  1.0
first event frame:  4

Stop  Id( 2 , None , None , None ) Loaded ( 735  records loaded in  0.09373617172241211 S )
Caching eventTimeLine
Contact  Id( 2 , None , None , None ) Loaded from cache ( 250  records. )
Stop  Id( 3 , None , None , None ) Loaded ( 650  records loaded in  0.0937650203704834 S )
Caching eventTimeLine
Contact  Id( 3 , None , None , None ) Loaded from cache ( 267  records. )
Stop  Id( 4 , None , None , None ) Loaded ( 655  records loaded in  0.0937502384185791 S )
Caching eventTimeLine
Contact  Id( 4 , None , None , None ) Loaded from cache ( 226  records. )
Stop isolated Stop in contact
Event Stop in contact created. eventNameWithId = Stop in contact idA:1 idB:None idC:None idD:None loadEvent: False
Number of event:  253
Mean length of event:  8.079051383399209
first event frame:  209
Keep previous entry.
Saving timeLine: Stop in contact ( 253 events )
Event Stop isolated created. eventNameWithId = Stop isolated idA:1 idB:None idC:None idD:None loadEvent: False
Number of event:  640
Mean

Approach contact
Event Approach contact created. eventNameWithId = Approach contact idA:3 idB:2 idC:None idD:None loadEvent: False
Number of event:  141
Mean length of event:  7.3120567375886525
first event frame:  768
Keep previous entry.
Saving timeLine: Approach contact ( 141 events )
Approach contact
Event Approach contact created. eventNameWithId = Approach contact idA:3 idB:4 idC:None idD:None loadEvent: False
Number of event:  119
Mean length of event:  7.159663865546219
first event frame:  197
Keep previous entry.
Saving timeLine: Approach contact ( 119 events )
Approach contact
Event Approach contact created. eventNameWithId = Approach contact idA:4 idB:1 idC:None idD:None loadEvent: False
Number of event:  94
Mean length of event:  5.3936170212765955
first event frame:  1451
Keep previous entry.
Saving timeLine: Approach contact ( 94 events )
Approach contact
Event Approach contact created. eventNameWithId = Approach contact idA:4 idB:2 idC:None idD:None loadEvent: False
Numb

seq oral geni - oral oral idA:4 idB:1 idC:None idD:None
Number of event:  1
Mean length of event:  60.0
first event frame:  9967
Keep previous entry.
Saving timeLine: seq oral geni - oral oral ( 1 events )
Event seq oral geni - oral oral created. eventNameWithId = seq oral geni - oral oral idA:4 idB:2 idC:None idD:None loadEvent: False
seq oral geni - oral oral idA:4 idB:2 idC:None idD:None
no event
Keep previous entry.
Saving timeLine: seq oral geni - oral oral ( 0 events )
Event seq oral geni - oral oral created. eventNameWithId = seq oral geni - oral oral idA:4 idB:3 idC:None idD:None loadEvent: False
seq oral geni - oral oral idA:4 idB:3 idC:None idD:None
Number of event:  1
Mean length of event:  17.0
first event frame:  13789
Keep previous entry.
Saving timeLine: seq oral geni - oral oral ( 1 events )
INSERT INTO LOG( process,version,date,tmin,tmax) VALUES ( 'Build Event Oral Side Sequence','0','2019-04-01 15:18:27','0','23890' );
Rebuild event finished.
[Chrono  <module 'lmtanal